In [255]:
import networkx as nx
import matplotlib.pyplot as plt
import community
import types
import pickle 
import pandas as pd
from networkx.algorithms.community.centrality import girvan_newman
from networkx.algorithms.community.kclique import k_clique_communities
from networkx.algorithms.community import greedy_modularity_communities

## Todos

TODO: The main thing is just to find a way to partition the graph into a reasonable number of clusters.

TODO: Try other options in networkx.algorithms.community. See https://networkx.github.io/documentation/stable/reference/algorithms/community.html

TODO: Recreate a graph, maybe in GEPHI, using whatever communinty structure we end up with

TODO: LSA / Semantic analysis

TODO: Add diagnostics showing how long some tasks take to run




## Load Graph
Read in the edge list, which contains all relevant information: source author -> target author, number of times cited

In [138]:
G_directed = nx.read_weighted_edgelist("./small_edge_list.csv", delimiter=",",create_using= nx.DiGraph())
G = nx.read_weighted_edgelist("./small_edge_list.csv", delimiter=",")

In [275]:
G_tiny = nx.read_weighted_edgelist("./tiny_edge_list.csv", delimiter=",")
G_tiny_directed = nx.read_weighted_edgelist("./tiny_edge_list.csv", delimiter=",", create_using= nx.DiGraph())
G_large_directed = nx.read_weighted_edgelist("./complete_edge_list.csv", delimiter=",",create_using= nx.DiGraph())
G_large = nx.read_weighted_edgelist("./complete_edge_list.csv", delimiter=",")

In [140]:
# Choose which graph to focus on
G_main = G_large_directed

In [12]:
# To convert files to graphml 
# nx.write_graphml(G_directed, "small_edge_list.graphml")

### Basic Stats

In [141]:
print(nx.info(G_main))
# print(nx.diameter(G))
# print(nx.average_neighbor_degree(G_large_directed))

Name: 
Type: DiGraph
Number of nodes: 9925
Number of edges: 55279
Average in degree:   5.5697
Average out degree:   5.5697


## Author statistics

In [142]:
# In degree
degreeTable = pd.DataFrame(G_main.in_degree, columns = ["Name","In Degree"])
sd = degreeTable.sort_values(by=['In Degree'], ascending=False)
sd.head(10)

,Name,In Degree
59,HUSSERL E,1163
229,HEIDEGGER M,834
1254,MERLEAUPONTY M,685
931,KANT I,378
2193,SARTRE J,317
2275,DERRIDA J,304
2753,HEGEL G,300
226,ZAHAVI D,289
2445,LEVINAS E,282
741,RICOEUR P,271


In [143]:
# Out-degree
degreeTable = pd.DataFrame(G_main.out_degree, columns = ["Name","Out Degree"])
sd = degreeTable.sort_values(by=['Out Degree'], ascending=False)
sd.head(10)

,Name,Out Degree
25,MORAN D,281
1339,FUCHS T,245
226,ZAHAVI D,223
434,FROESE T,196
440,GALLAGHER S,195
1071,SCHWITZGEBEL E,143
163,HEINAMAA S,142
10,DEPRAZ N,140
200,SASS L,134
119,WRATHALL M,131


In [144]:
# Hubs and Authorities
h,a=nx.hits(G_main)

# First hubs
hubTable =  pd.DataFrame.from_dict(h,orient='index', columns=["Hubitude"])
sd = hubTable.sort_values(by=['Hubitude'], ascending=False)
sd.head(20)

,Hubitude
ZAHAVI D,0.012030
DEPRAZ N,0.007946
MORAN D,0.007362
CROWELL S,0.005895
HEINAMAA S,0.005108
FERENCZ F,0.004465
GALLAGHER S,0.004223
SHEETS J,0.003789
FUCHS T,0.003680
HARTIMO M,0.003632


In [145]:
# Then authorities
authTable =  pd.DataFrame.from_dict(a,orient='index', columns=["Authority"])
sd = authTable.sort_values(by=['Authority'], ascending=False)
sd.head(20)

,Authority
HUSSERL E,0.058220
HEIDEGGER M,0.024142
MERLEAUPONTY M,0.021066
ZAHAVI D,0.013648
SARTRE J,0.008362
DERRIDA J,0.008345
LEVINAS E,0.008026
GALLAGHER S,0.007909
KANT I,0.006420
DREYFUS H,0.006238


In [146]:
# Eigenvector centrality
centrality = nx.eigenvector_centrality(G_directed)
ec_table =  pd.DataFrame.from_dict(centrality,orient='index', columns=["Centrality"])
ect = ec_table.sort_values(by=['Centrality'], ascending=False)
ect.head(20)

,Centrality
HUSSERL E,0.405200
HEIDEGGER M,0.267859
MERLEAUPONTY M,0.241819
ZAHAVI D,0.157583
KANT I,0.143643
SARTRE J,0.132890
FINK E,0.125441
GALLAGHER S,0.116983
DREYFUS H,0.113294
LEVINAS E,0.111870


## Community Detection

See https://networkx.github.io/documentation/stable/reference/algorithms/community.html

## Helper Methods

In [212]:
# Currently only prints top level clusters
# todo: recursively print all sub-clusters
def print_clusters(graph, clusters):
    print("Graph has ",len(clusters), "clusters")
    for i,p in enumerate(clusters):
        print("Cluster",(i+1), "has", len(p), "members")

In [194]:
# Converts nested generators to a list of lists
# The list can then be saved via pickle
# I suspect _extremely_ slow for large lists
# From https://stackoverflow.com/questions/36726129/convert-nested-iterables-to-list
def unfold(iterable):
    ret = []
    for element in iterable:
        if isinstance(element, types.GeneratorType):
            ret.append(unfold(element))
        else:
            ret.append(element)
    return ret

### Louvain

https://python-louvain.readthedocs.io/en/latest/api.html

Appears to be deterministic with default settings

Does not seem to work with directed graphs

I think it's using weights but not sure

In [262]:
partition = community.generate_dendrogram(G_large)

In [263]:
print_clusters(G_large, partition)

Graph has  3 clusters
Cluster 1 has 9925 members
Cluster 2 has 705 members
Cluster 3 has 51 members


In [157]:
# Save / re-open a partition
# pickle.dump(partition, open("partition_louvain.obj", "wb"))
# test = pickle.load(open("partition_louvain.obj", "rb"))

### Girvan Newman

Takes a long time...

In [276]:
partition_gn = girvan_newman(G_tiny_directed)

In [277]:
clusters_gn = unfold(partition_gn)

In [278]:
# print_clusters(G_tiny, clusters_gn) # Something is obviously wrong!

Graph has  178 clusters
Cluster 1 has 2 members
Cluster 2 has 3 members
Cluster 3 has 4 members
Cluster 4 has 5 members
Cluster 5 has 6 members
Cluster 6 has 7 members
Cluster 7 has 8 members
Cluster 8 has 9 members
Cluster 9 has 10 members
Cluster 10 has 11 members
Cluster 11 has 12 members
Cluster 12 has 13 members
Cluster 13 has 14 members
Cluster 14 has 15 members
Cluster 15 has 16 members
Cluster 16 has 17 members
Cluster 17 has 18 members
Cluster 18 has 19 members
Cluster 19 has 20 members
Cluster 20 has 21 members
Cluster 21 has 22 members
Cluster 22 has 23 members
Cluster 23 has 24 members
Cluster 24 has 25 members
Cluster 25 has 26 members
Cluster 26 has 27 members
Cluster 27 has 28 members
Cluster 28 has 29 members
Cluster 29 has 30 members
Cluster 30 has 31 members
Cluster 31 has 32 members
Cluster 32 has 33 members
Cluster 33 has 34 members
Cluster 34 has 35 members
Cluster 35 has 36 members
Cluster 36 has 37 members
Cluster 37 has 38 members
Cluster 38 has 39 members
Clust

### K Clique Communities

In [271]:
# Appears to be deterministic
clusters_kc = k_clique_communities(G_large,4)

In [272]:
list_kc = unfold(clusters_kc)

In [274]:
# print_clusters(G_tiny, list_kc) 

In [69]:
# Cluster with 873 members
#                Name  In Degree
# 428       HUSSERL E        535
# 829     HEIDEGGER M        326
# 225  MERLEAUPONTY M        303


## Greedy 

In [257]:
# Appears to be deterministic
clusters_greedy = k_clique_communities(G_large, 5)

In [258]:
list_greedy = unfold(clusters_greedy)

In [259]:
print_clusters(G_large, list_greedy) 

Graph has  50 clusters
Cluster 1 has 918 members
Cluster 2 has 5 members
Cluster 3 has 5 members
Cluster 4 has 5 members
Cluster 5 has 5 members
Cluster 6 has 5 members
Cluster 7 has 11 members
Cluster 8 has 6 members
Cluster 9 has 5 members
Cluster 10 has 5 members
Cluster 11 has 6 members
Cluster 12 has 6 members
Cluster 13 has 6 members
Cluster 14 has 5 members
Cluster 15 has 19 members
Cluster 16 has 5 members
Cluster 17 has 7 members
Cluster 18 has 5 members
Cluster 19 has 5 members
Cluster 20 has 5 members
Cluster 21 has 6 members
Cluster 22 has 5 members
Cluster 23 has 5 members
Cluster 24 has 5 members
Cluster 25 has 5 members
Cluster 26 has 7 members
Cluster 27 has 5 members
Cluster 28 has 7 members
Cluster 29 has 5 members
Cluster 30 has 5 members
Cluster 31 has 6 members
Cluster 32 has 5 members
Cluster 33 has 5 members
Cluster 34 has 5 members
Cluster 35 has 5 members
Cluster 36 has 5 members
Cluster 37 has 5 members
Cluster 38 has 5 members
Cluster 39 has 5 members
Cluster

## Draw graph

In [ ]:
# nx.draw(G, node_size = 10, with_labels=True, font_size = 12)

### Archived functions 
There may be some useful code fragments here

Delete ASAP following the principle of commented out code is gross and should not be allowed to accumulate

In [160]:
def print_clusters(graph, clusters):
    print("Graph has ",len(clusters), "clusters")
    for i,p in enumerate(clusters):
        print("Cluster",(i+1), "has", len(p), "members")
#         if (len(p) > 0):
#             for i2,p2 in enumerate(p):
#                 print("\tSubCluster",(i2+1), "has", len(p2), "members")
                
#         subgraph = graph.subgraph([i for i in list(p)])
#         print(subgraph)
#         table = pd.DataFrame(graph.degree, columns = ["Name","Degree"])
#         table = table.sort_values(by=['Degree'], ascending=False)
#         print(st.head(3))
#         print()

In [ ]:
def print_community(community, hasmembers=True):
    print(len(community))
    if(hasmembers):
        for comm in community:
            print(comm)
    else:
        print(community)